In [1]:
import sys
sys.path.insert(0, '..')

from src.database import Database
from src.network_analysis import NetworkAnalyzer
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

import plotly.express as px
import pandas as pd

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

2025-10-14 09:56:26,958 - INFO - NumExpr defaulting to 8 threads.


In [2]:
courses_df = pd.read_csv('data/csv_exports/course_offerings_per_semester.csv')
courses_df

C:\Users\ashto\AppData\Local\Temp\ipykernel_36100\3739843695.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  courses_df = pd.read_csv('data/csv_exports/course_offerings_per_semester.csv')


,department_code,department_name,course_number,course_title,full_code,term,year,section,crn,enrollment,capacity,waitlist
0,AGBI,NaN,10,Intro Biochemistry,AGBI 010,Spring,1995,A,13159,12,999,NaN
1,AGBI,NaN,191,Biochem Nucleic Acids,AGBI 191,Spring,1995,A,13160,9,30,NaN
2,AGBI,NaN,195,Special Topics,AGBI 195,Spring,1995,A,11000,0,10,NaN
3,AGBI,NaN,195,Special Topics,AGBI 195,Spring,1995,B,11001,0,5,NaN
4,AGBI,NaN,198,Undergraduate Research,AGBI 198,Spring,1995,A,11002,5,999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
251722,SWSS,NaN,6990,Disability Rights & TSW,SWSS 6990,Summer,2025,Z3,61788,3,4,NaN
251723,THE,NaN,1500,Dramatic Analysis,THE 1500,Summer,2025,OL1,61879,17,40,NaN
251724,WFB,NaN,2310,Field Ornithology,WFB 2310,Summer,2025,Z1,61302,30,30,NaN
251725,WFB,NaN,3240,Conservation Biology,WFB 3240,Summer,2025,OL1,61107,11,35,NaN


# Raw titles, unique count

In [3]:
courses_df['course_title'].value_counts()

course_title
Doctoral Dissertation Research    8448
FTReg Animal Science              8002
HTReg Animal Science              7805
LHReg Animal Science              7697
Master's Thesis Research          7326
                                  ... 
Int'l Migration & US Society         1
Self and Society                     1
Global Health Crises                 1
Ecological Restor Politics           1
Literature 20/21st Centuries         1
Name: count, Length: 6812, dtype: int64

In [4]:
# Get course counts
course_counts = courses_df['course_title'].value_counts().reset_index()
course_counts.columns = ['course_title', 'count']
course_counts['rank'] = course_counts.index + 1  # add rank (1-based)

# Create Plotly bar chart
fig = px.bar(
    course_counts.head(250),  # show top 50 courses (adjust as needed)
    x='rank',
    y='count',
    hover_name='course_title',
    title='Most Common Course Titles (Ranked)',
)

# Customize axis
fig.update_layout(
    xaxis_title='Rank',
    yaxis_title='Number of Courses',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(0, 250, 25)),  # show every 5th rank label
        ticktext=[str(i) for i in range(0, 250, 25)],
    ),
    template='plotly_white',  # optional aesthetic
)

fig.show()

In [5]:
print(course_counts[:30])

                      course_title  count  rank
0   Doctoral Dissertation Research   8448     1
1             FTReg Animal Science   8002     2
2             HTReg Animal Science   7805     3
3             LHReg Animal Science   7697     4
4         Master's Thesis Research   7326     5
5                   Animal Science   5354     6
6                   Special Topics   4325     7
7          Private Lessons:Bassoon   2424     8
8                Independent Study   2352     9
9                Club Sports: Crew   2220    10
10             Readings & Research   2004    11
11              Written Expression   1907    12
12          Undergraduate Research   1842    13
13          Introductory Chemistry   1737    14
14                      Internship   1600    15
15               Exploring Biology   1516    16
16          Teaching Assistantship   1422    17
17      Doctoral Dissertation Rsch   1336    18
18               Organic Chemistry   1308    19
19               Women's X-Country   117

In [6]:
courses_df[['Chemistry' in x for x in courses_df['course_title']]]

,department_code,department_name,course_number,course_title,full_code,term,year,section,crn,enrollment,capacity,waitlist
419,CHEM,NaN,23,Outline of General Chemistry,CHEM 023,Spring,1995,A,11157,75,190,NaN
420,CHEM,NaN,23,Outline of General Chemistry,CHEM 023,Spring,1995,WD,17087,1,999,NaN
421,CHEM,NaN,23,Outline of General Chemistry,CHEM 023,Spring,1995,WD1,17101,1,999,NaN
422,CHEM,NaN,25,Outline of General Chemistry,CHEM 025,Spring,1995,A,11176,25,20,NaN
427,CHEM,NaN,32,Intro Chemistry,CHEM 032,Spring,1995,A,15162,131,180,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
250207,CHEM,NaN,1450,General Chemistry 2,CHEM 1450,Summer,2025,Z1,61916,40,60,NaN
250208,CHEM,NaN,1580,Intro Organic Chemistry w/lab,CHEM 1580,Summer,2025,Z1,61917,28,40,NaN
250209,CHEM,NaN,1582,Intro Organic Chemistry,CHEM 1582,Summer,2025,OL1,61918,2,5,NaN
250210,CHEM,NaN,2580,Organic Chemistry 1,CHEM 2580,Summer,2025,Z1,61919,26,60,NaN


# Extract individual words

In [7]:
import pandas as pd
import re
from collections import Counter

# Example: using your existing df from joined query
titles = courses_df['course_title'].fillna('')

# Normalize text
cleaned = (
    titles
    .str.lower()                       # lowercase
    .str.replace(':', ' ', regex=False) # replace ':' with space
    .str.replace('[^a-z0-9\s]', '', regex=True)  # remove punctuation
)

# Tokenize into words
all_words = [word for title in cleaned for word in title.split()]

# Count frequency
word_counts = Counter(all_words)

# Convert to DataFrame
word_df = (
    pd.DataFrame(word_counts.items(), columns=['word', 'count'])
    .sort_values(by='count', ascending=False)
    .reset_index(drop=True)
)
word_df['rank'] = word_df.index + 1

# Show top results
print(word_df.head(20))

            word  count  rank
0        science  30526     1
1         animal  29592     2
2       research  25621     3
3             of  17583     4
4             in  12222     5
5       doctoral  10343     6
6   dissertation  10323     7
7              i  10315     8
8         thesis   9313     9
9        masters   8764    10
10         ftreg   8002    11
11         intro   7990    12
12         htreg   7805    13
13         lhreg   7697    14
14       special   7548    15
15        topics   7058    16
16       lessons   6865    17
17     chemistry   6488    18
18            ii   6361    19
19            to   6094    20


In [8]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming word_df (or filtered_df) is already created
# with columns: 'word', 'count', 'rank'

df = word_df.copy()

# Sort by count descending just to be sure
df = df.sort_values(by='count', ascending=False).reset_index(drop=True)
df['rank'] = df.index + 1

# --- Compute CCDF ---
sorted_counts = np.sort(df['count'])[::-1]
unique_counts, freq = np.unique(sorted_counts, return_counts=True)
ccdf = 1.0 - np.cumsum(freq) / np.sum(freq)

# --- Create subplots ---
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Zipf Rank Plot', 'CCDF of Word Frequencies'),
    horizontal_spacing=0.15
)

# --- Left: Zipf rank plot ---
fig.add_trace(
    go.Scatter(
        x=df['rank'],
        y=df['count'],
        mode='markers',
        name='Word frequency',
        marker=dict(size=6, opacity=0.7)
    ),
    row=1, col=1
)

# --- Right: CCDF plot ---
fig.add_trace(
    go.Scatter(
        x=unique_counts,
        y=ccdf,
        mode='markers',
        name='CCDF',
        marker=dict(size=6, opacity=0.7)
    ),
    row=1, col=2
)

# --- Layout ---
fig.update_xaxes(
    title_text="Rank",
    type="log",
    row=1, col=1
)
fig.update_yaxes(
    title_text="Frequency",
    type="log",
    row=1, col=1
)

fig.update_xaxes(
    title_text="Count",
    type="log",
    row=1, col=2
)
fig.update_yaxes(
    title_text="P(X ≥ x)",
    type="log",
    row=1, col=2
)

fig.update_layout(
    # title_text="Word Frequency Distributions (Zipf & CCDF)",
    template="plotly_white",
    showlegend=False,
    height=500,
    width=1000
)

fig.show()


# Allotax setup

In [9]:
def save_courseword_file(df, start_year, end_year, folder="data/course_data"):
    """
    Saves unique word counts for course data to standardized CSV file.
    df: DataFrame with columns ['word', 'count']
    """
    os.makedirs(folder, exist_ok=True)
    
    # Sort by frequency
    df = df.sort_values("count", ascending=False).reset_index(drop=True)
    df["k"] = df.index + 1
    df["Nk"] = df["count"]
    
    # Summary values
    totalunique = df["word"].nunique()
    totalcount = df["count"].sum()
    
    # Prepare result DataFrame
    result = pd.DataFrame({
        "types": df["word"],
        "counts": df["count"],
        "totalunique": totalunique,
        "probs": df["count"] / totalcount
    })
    
    # Save file
    filename = os.path.join(folder, f"coursewords-{start_year}-{end_year}.csv")
    result.to_csv(filename, index=False)
    print(f"✅ Saved {filename} with {len(df)} entries.")

# Example usage:
# Assuming you’ve already created these two filtered dataframes:
# word_counts_1995_2000 and word_counts_2020_2025


import sqlite3
import pandas as pd
import re
from collections import Counter

# Path to your database
db_path = "../data/uvm_courses.db"

# ----------------------------------------------------------
# 1️⃣ Load tables
# ----------------------------------------------------------
with sqlite3.connect(db_path) as conn:
    courses_df = pd.read_sql("SELECT * FROM courses", conn)
    offerings_df = pd.read_sql("SELECT * FROM course_offerings", conn)

# Merge course titles with years
merged = offerings_df.merge(courses_df, left_on="course_id", right_on="id", how="left")

# ----------------------------------------------------------
# 2️⃣ Function to clean and count words
# ----------------------------------------------------------
def count_course_words(df):
    all_words = []
    for title in df["course_title"].dropna():
        title = re.sub(r"[:;,.!?()'\"]", " ", title.lower())  # remove punctuation
        tokens = re.findall(r"[a-zA-Z]+", title)              # keep only alphabetic tokens
        all_words.extend(tokens)
    return pd.DataFrame(Counter(all_words).most_common(), columns=["word", "count"])

# ----------------------------------------------------------
# 3️⃣ Filter by year range
# ----------------------------------------------------------
def get_word_counts_by_year_range(df, start, end):
    subset = df[(df["year"] >= start) & (df["year"] <= end)]
    return count_course_words(subset)

word_counts_1995_2000 = get_word_counts_by_year_range(merged, 1995, 2000)
word_counts_2020_2025 = get_word_counts_by_year_range(merged, 2020, 2025)

# ----------------------------------------------------------
# 4️⃣ Sort, rank, and add metadata
# ----------------------------------------------------------
def prepare_word_rank(df):
    df = df.sort_values("count", ascending=False).reset_index(drop=True)
    df["k"] = df.index + 1
    df["Nk"] = df["count"]
    df["totalunique"] = df["word"].nunique()
    df["probs"] = df["count"] / df["count"].sum()
    return df

word_counts_1995_2000 = prepare_word_rank(word_counts_1995_2000)
word_counts_2020_2025 = prepare_word_rank(word_counts_2020_2025)

# ----------------------------------------------------------
# 5️⃣ Save to CSV
# ----------------------------------------------------------
import os
folder = "data/course_data"
os.makedirs(folder, exist_ok=True)

word_counts_1995_2000.to_csv(f"{folder}/coursewords-1995-2000.csv", index=False)
word_counts_2020_2025.to_csv(f"{folder}/coursewords-2020-2025.csv", index=False)

print("✅ Files saved to data/course_data/")


save_courseword_file(word_counts_1995_2000, 1995, 2000)
save_courseword_file(word_counts_2020_2025, 2020, 2025)


✅ Files saved to data/course_data/
✅ Saved data/course_data\coursewords-1995-2000.csv with 2132 entries.
✅ Saved data/course_data\coursewords-2020-2025.csv with 3126 entries.


In [25]:
import pandas as pd
import plotly.graph_objects as go

# Load data
df = pd.read_csv('data/csv_exports/course_offerings_per_semester.csv')

# Group by year and department_code, summing enrollment
enrollment_per_year_dept = (
    df.groupby(['year', 'department_code'])['enrollment']
    .sum()
    .reset_index()
)

# Create figure
fig = go.Figure()

# Add a trace per department
for dept_code in enrollment_per_year_dept['department_code'].unique():
    dept_data = enrollment_per_year_dept[
        enrollment_per_year_dept['department_code'] == dept_code
    ]
    fig.add_trace(go.Scatter(
        x=dept_data['year'],
        y=dept_data['enrollment'],
        mode='lines+markers',
        name=dept_code
    ))

# Customize layout
fig.update_layout(
    title='Enrollment per Year by Department',
    xaxis_title='Year',
    yaxis_title='Total Enrollment',
    template='plotly_white',
    # hovermode='x unified',
    height=1000,
    width=1500,
)
fig.write_html('enrollment_by_department.html')
# fig.show()


C:\Users\ashto\AppData\Local\Temp\ipykernel_36100\4125597756.py:5: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
# Load data
df = pd.read_csv('data/csv_exports/course_offerings_per_semester.csv')

# Filter only Spring and Fall semesters (assuming 'term' column has 'Spring', 'Fall', etc.)
df = df[df['term'].isin(['Spring', 'Fall'])]

# 🔹 Choose a department name (edit this to pick your department)
selected_dept = "GEOL"

# Filter for selected department
dept_df = df[df['department_code'] == selected_dept]

# Group by year, term, and course
enrollment_by_course_sem = (
    dept_df.groupby(['year', 'term', 'full_code'])['enrollment']
    .sum()
    .reset_index()
)

# Combine year and term for x-axis label
enrollment_by_course_sem['semester_label'] = (
    enrollment_by_course_sem['year'].astype(str) + " " + enrollment_by_course_sem['term']
)

# Sort semester labels chronologically
enrollment_by_course_sem = enrollment_by_course_sem.sort_values(['year', 'term'])

# Create figure
fig = go.Figure()

# Add a line for each course
for course_code in enrollment_by_course_sem['full_code'].unique():
    course_data = enrollment_by_course_sem[
        enrollment_by_course_sem['full_code'] == course_code
    ]
    fig.add_trace(go.Scatter(
        x=course_data['semester_label'],
        y=course_data['enrollment'],
        mode='lines+markers',
        name=course_code
    ))

# Customize layout
fig.update_layout(
    title=f'{selected_dept} Enrollment by Course (Spring/Fall)',
    xaxis_title='Semester',
    yaxis_title='Enrollment',
    template='plotly_white',
    height=800,
    width=1400,
    # hovermode='x unified'
)

# Export to HTML
fig.write_html(f'enrollment_by_course_{selected_dept}.html')

# Optionally show
# fig.show()

C:\Users\ashto\AppData\Local\Temp\ipykernel_36100\2385107085.py:2: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.

